In [28]:
# imports

import pandas as pd
import yfinance as yf
import yahoofinancials as YahooFinancials
from datetime import date, datetime, timedelta

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense , Dropout , LSTM, CuDNNLSTM

In [29]:
# Define the lookup for the stock (name, price, symbol)

import requests
import urllib.parse


api_key = 'pk_35ab12ce73ea4cf38d7641e8d3b01de7'

def lookup(symbol, api_key="pk_35ab12ce73ea4cf38d7641e8d3b01de7"):
    """Look up quote for symbol."""

    # Contact API
    try:
        response = requests.get(
            f"https://cloud.iexapis.com/stable/stock/{urllib.parse.quote_plus(symbol)}/quote?token={api_key}")
        response.raise_for_status()
    except requests.RequestException:
        return None

    # Parse response
    try:
        quote = response.json()
        return {
                "name": quote["companyName"],
                "price": float(quote["latestPrice"]),
                "symbol": quote["symbol"]
                }
    except (KeyError, TypeError, ValueError):
        return None


In [30]:
# input the stock symbol

symbol = input("Enter a stock symbol: ")

In [31]:
# get todays date

today = date.today()
print(today)

2022-04-07


In [32]:
# print the lookup for the sumbol

print(lookup(symbol))

{'name': 'Apple Inc', 'price': 172.72, 'symbol': 'AAPL'}


In [33]:
# date mathematics
# another import line because the code doesn't work without it

from datetime import datetime, timedelta

timedifference = input("Enter a time difference in months: ")
timedifference = int(timedifference)

# timedifference is the number of months
timediffdays = timedifference * 30
# timediffdays is the number of days

startdate = datetime.today() - timedelta(days=timediffdays)
print(startdate)


2022-03-08 22:03:48.848033


In [38]:
# gets data from yahoo finance from a set date to today

stock_df = yf.download(symbol, 
                      start=startdate, 
                      end=today, 
                      progress=False,
)
stock_df.drop(columns=['High', 'Low', 'Close', 'Adj Close', 'Volume'], inplace=True)
stock_df.head()

,Open
Date,
2022-03-08,158.820007
2022-03-09,161.479996
2022-03-10,160.199997
2022-03-11,158.929993
2022-03-14,151.449997


In [35]:
stock_values_list = stock_df.values.tolist()
print(stock_values_list)

# list of lists of data in the form [[open, high, low, close, volume, adj close]]

[[158.82000732421875, 162.8800048828125, 155.8000030517578, 157.44000244140625, 157.44000244140625, 131148300.0], [161.47999572753906, 163.41000366210938, 159.41000366210938, 162.9499969482422, 162.9499969482422, 91454900.0], [160.1999969482422, 160.38999938964844, 155.97999572753906, 158.52000427246094, 158.52000427246094, 105342000.0], [158.92999267578125, 159.27999877929688, 154.5, 154.72999572753906, 154.72999572753906, 96836300.0], [151.4499969482422, 154.1199951171875, 150.10000610351562, 150.6199951171875, 150.6199951171875, 108732100.0], [150.89999389648438, 155.57000732421875, 150.3800048828125, 155.08999633789062, 155.08999633789062, 92964300.0], [157.0500030517578, 160.0, 154.4600067138672, 159.58999633789062, 159.58999633789062, 102300200.0], [158.61000061035156, 161.0, 157.6300048828125, 160.6199951171875, 160.6199951171875, 75615400.0], [160.50999450683594, 164.47999572753906, 159.75999450683594, 163.97999572753906, 163.97999572753906, 123351200.0], [163.50999450683594, 1

In [36]:
# only keep the open price
# we will predict the prices based on opening prices

stock_open_list = []

for valueset in stock_values_list:
    stock_open_list.append(valueset[0])

print(stock_open_list)

[158.82000732421875, 161.47999572753906, 160.1999969482422, 158.92999267578125, 151.4499969482422, 150.89999389648438, 157.0500030517578, 158.61000061035156, 160.50999450683594, 163.50999450683594, 165.50999450683594, 167.99000549316406, 171.05999755859375, 173.8800048828125, 172.1699981689453, 176.69000244140625, 178.5500030517578, 177.83999633789062, 174.02999877929688, 174.57000732421875, 177.5, 172.36000061035156]


In [37]:

mnist = tf.keras.datasets.mnist
mnist = stock_df
(x_train,y_train),(x_test , y_test) = mnist.load_data()

x_train = x_train/255.0
y_train = y_train / 255.0
model=Sequential()

model.add(CuDNNLSTM(128 , input_shape=(x_train.shape[1:]), return_sequences = True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))

model.add(Dense(32 , activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation = 'softmax'))
opt = tf.keras.optimizers.Adam (lr=1e-3 , decay=1e-5)

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = opt, metrics =['accuracy'])

model.fit(x_train , y_train , epochs = 3 , validation_data = (x_test,y_test))


AttributeError: 'DataFrame' object has no attribute 'load_data'